In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark
!tar -xzf sql-data.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
import random

spark = SparkSession.builder.appName("YourTest").master("local[2]").config('spark.ui.port', random.randrange(4000,5000)).getOrCreate()

In [ ]:
def load_dataset_and_set_views():
    global supplier, orders, customer, partsupp, nation, part
    
    supplier_raw = spark.read.csv("supplier.tbl",sep='|',inferSchema=True).drop("_c7")
    supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
    supplier.createOrReplaceTempView("supplier")
    
    order_raw = spark.read.csv("orders.tbl",sep='|',inferSchema=True).drop("_c9")
    orders = order_raw.toDF("OrderKey","CustKey","OrderStatus","TotalPrice","OrderDate","Order-Pri","Clerk", "Ship-Pri", "Comment").cache()
    orders.createOrReplaceTempView("orders")

    customer_raw = spark.read.csv("customer.tbl",sep='|',inferSchema=True).drop("_c8")
    customer = customer_raw.toDF("CustKey","Name","Address","NationKey","Phone","AcctBal","MktSegment", "Comment").cache()
    customer.createOrReplaceTempView("customer")
    
    partsupp_raw = spark.read.csv("partsupp.tbl",sep='|',inferSchema=True).drop("_c5")
    partsupp = partsupp_raw.toDF("PartKey","SuppKey","AvailQty","SupplyCost","Comment").cache()
    partsupp.createOrReplaceTempView("partsupp")
    
    nation_raw = spark.read.csv("nation.tbl",sep='|',inferSchema=True).drop("_c4")
    nation = nation_raw.toDF("NationKey","Name","RegionKey", "Comment").cache()
    nation.createOrReplaceTempView("nation")
    
    part_raw = spark.read.csv("part.tbl",sep='|',inferSchema=True).drop("_c9")
    part = part_raw.toDF("PartKey","Name","Mfgr","Brand","Type","Size","Container", "RetailPrice", "Comment").cache()
    part.createOrReplaceTempView("part")

In [ ]:
def five_highest_totalprice_orders_sql():

    five_highest_totalprice_orders = spark.sql("select OrderKey, OrderDate, TotalPrice from Orders order by TotalPrice desc limit 5")
    return five_highest_totalprice_orders

In [ ]:
def five_highest_totalprice_orders_dtf():

    five_highest_totalprice_orders = orders.orderBy(orders.TotalPrice.desc()).select('OrderKey', 'OrderDate', 'TotalPrice')
    return five_highest_totalprice_orders.limit(5)

In [ ]:
def cust_most_recent_order_sql(custkey):

    cust_most_recent_order = spark.sql("select Name, OrderDate, TotalPrice from Orders left\
     join customer C on Orders.CustKey = C.CustKey where C.CustKey = {} order by OrderDate desc limit 1".format(custkey))
      
    return cust_most_recent_order

In [ ]:
def cust_most_recent_order_dtf(custkey):

    cust_most_recent_order = orders.filter("CustKey = {}".format(custkey))\
                                   .join(customer, orders.CustKey == customer.CustKey,"left")\
                                   .select('Name', 'OrderDate', 'TotalPrice')\
                                   .orderBy(orders.OrderDate.desc())

    return cust_most_recent_order.limit(1)

In [ ]:
def distinct_supplied_parts(nname):

    distinct_supplied_parts = spark.sql('select count(distinct P.PartKey) from partsupp P left join supplier S\
     on S.SuppKey = P.SuppKey where S.NationKey = (select NationKey from nation where Name = "{}")'.format(nname))

    return distinct_supplied_parts.head()[0]

In [ ]:
def count_suppliers_brand_per_nation(bname):

    count_suppliers_brand_per_nation = part.filter("Brand = '{}'".format(bname))\
                                           .join(partsupp, part.PartKey == partsupp.PartKey,"left")\
                                           .join(supplier, partsupp.SuppKey == supplier.SuppKey,"left")\
                                           .join(nation, supplier.NationKey == nation.NationKey, "left")\
                                           .groupBy(nation.Name, partsupp.SuppKey)\
                                           .count()\
                                           .groupBy(nation.Name)\
                                           .count()\
                                           .orderBy(nation.Name)
    
    return count_suppliers_brand_per_nation

In [ ]:
def order_number_per_customer_nation(nname):

    order_number_per_customer_nation = spark.sql('select year(O.OrderDate) as year, count(O.OrderKey) as num_orders from orders O left join customer C\
    on C.CustKey = O.CustKey where C.NationKey = (select NationKey from nation where Name = "{}") group by year(O.OrderDate)\
    order by num_orders desc'.format(nname))

    return order_number_per_customer_nation